In [4]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
from collections import defaultdict
from nltk.util import ngrams
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import gutenberg
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
# Load the corpus
corpus = gutenberg.raw()

In [ ]:
# Tokenize the corpus
tokens = word_tokenize(corpus)

In [ ]:
# Create a n-grams model
n = 3
ngrams_model = ngrams(tokens, n)


In [ ]:
# Create a frequency distribution of the n-grams using parallel processing
def count_ngrams(ngrams):
    fdist = FreqDist(ngrams)
    return fdist


In [ ]:
with ProcessPoolExecutor() as executor:
    fdist = defaultdict(int)
    for fdist_chunk in executor.map(count_ngrams, ngrams_model, chunksize=1000):
        for k,v in fdist_chunk.items():
            fdist[k] += v


In [ ]:
# Function to predict the next word
def predict_next_word(word):
    possible_ngrams = [w for w in fdist if w[:n-1] == word[-n+1:]]
    total = sum(fdist[w] for w in possible_ngrams)
    next_words = defaultdict(int)
    for w in possible_ngrams:
        next_words[w[-1]] = fdist[w]/total
    return max(next_words, key=next_words.get)


In [ ]:
# Function to predict the list of probable next words
def predict_next_word_list(word):
    possible_ngrams = [w for w in fdist if w[:n-1] == word[-n+1:]]
    total = sum(fdist[w] for w in possible_ngrams)
    next_words = defaultdict(int)
    for w in possible_ngrams:
        next_words[w[-1]] = fdist[w]/total
    return sorted(next_words, key=next_words.get,reverse=True)


In [ ]:
# Save the model to a file
import pickle
with open('ngrams_model.pickle', 'wb') as handle:
    pickle.dump(fdist, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load the model from a file
with open('ngrams_model.pickle', 'rb') as handle:
    fdist = pickle.load(handle)

In [ ]:
# Example usage
print(predict_next_word(('the','world')))
print(predict_next_word_list(('the','world')))
